<a href="https://colab.research.google.com/github/HenriqueCCdA/ImersaoDados/blob/main/%5CAula%5CImers%C3%A3oDados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 1

In [ ]:
import pandas as pd

In [ ]:
url_dados = "https://github.com/alura-cursos/imersaodados3/blob/main/dados/dados_experimentos.zip?raw=true"

dados = pd.read_csv(url_dados, compression='zip')

In [ ]:
dados.head()

In [ ]:
dados.shape

In [ ]:
dados['tratamento'].unique()

In [ ]:
dados['tempo'].unique()

In [ ]:
dados['dose'].unique()

In [ ]:
dados['g-0'].unique()

In [ ]:
dados['droga'].unique().shape

In [ ]:
dados['tratamento'].value_counts()

In [ ]:
dados['tratamento'].value_counts(normalize = True)

In [ ]:
dados['tratamento'].value_counts()

In [ ]:
dados['dose'].value_counts()

In [ ]:
dados['dose'].value_counts(normalize = True)

In [ ]:
dados['tratamento'].value_counts().plot.pie()

In [ ]:
dados['tempo'].value_counts().plot.pie()

In [ ]:
dados['tempo'].value_counts().plot.bar()

In [ ]:
dados_filtrados = dados[dados['g-0'] > 0]
dados_filtrados.head()

In [ ]:
new_columns = []
for word in dados.columns:
  if "-" in word:
    new_columns.append(word.replace("-", ""))
  else:
    new_columns.append(word)

dados.columns = new_columns

dados.head()

In [ ]:
dados.query("g0 > 0")

#Aula 2

In [ ]:
mapa = {'droga' : 'compostos'}
dados.rename(columns=mapa, inplace=True)
dados.head()

In [ ]:
cod_compostos = dados['compostos'].value_counts().index[:5]

In [ ]:
cod_compostos

In [ ]:
dados.query("compostos in @cod_compostos")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set()
plt.figure(figsize=(8, 6))

ax = sns.countplot(x = 'compostos', data = dados.query("compostos in @cod_compostos"))
ax.set_title('Top 5 compostos')
plt.show()

In [ ]:
dados['g0'].hist(bins = 100)

In [ ]:
dados['g19'].hist(bins = 100)

In [ ]:
dados.describe()

In [ ]:
dados.loc[:,'g0':'g771'].describe()

In [ ]:
dados.loc[:,'g0':'g771'].describe().T['mean'].hist(bins=50)

In [ ]:
dados.loc[:,'g0':'g771'].describe().T['min'].hist(bins=50)

In [ ]:
dados.loc[:,'g0':'g771'].describe().T['max'].hist(bins=50)

In [ ]:
dados.loc[:,'c0':'c99'].describe().T['mean'].hist(bins=50)

In [ ]:
sns.boxplot(x='g0', data=dados)

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(y='g0', x ='tratamento' , data=dados)

# Aula 3

In [ ]:
pd.crosstab(dados['dose'], dados['tempo'])

In [ ]:
pd.crosstab([dados['dose'], dados['tempo']], dados['tratamento'])

In [ ]:
pd.crosstab([dados['dose'], dados['tempo']], dados['tratamento'], normalize='index')

In [ ]:
pd.crosstab([dados['dose'], dados['tempo']], dados['tratamento'],values=dados['g0'], aggfunc = 'mean')

In [ ]:
sns.scatterplot(x='g0', y = 'g3', data=dados)

In [ ]:
sns.scatterplot(x='g0', y = 'g8', data=dados)

In [ ]:
sns.lmplot(data=dados, x = 'g0', y='g8', line_kws={'color':'red'})

In [ ]:
sns.lmplot(data=dados, x = 'g0', y='g8', line_kws={'color':'red'}, col='tratamento', row='tempo')

In [ ]:
dados.loc[:,'g0':'g771'].corr()

In [ ]:
# Compute the correlation matrix
corr = dados.loc[:,'c0':'c50'].corr()

In [ ]:
import numpy as np

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.8, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

#Aula 4


In [ ]:
url_dados_resultados = "https://github.com/alura-cursos/imersaodados3/blob/main/dados/dados_resultados.csv?raw=true"

dados_resultados = pd.read_csv(url_dados_resultados)
dados_resultados.head()

In [ ]:
dados_resultados['acat_inhibitor'].unique()

In [ ]:
dados_resultados['acetylcholine_receptor_agonist'].unique()

In [ ]:
dados_resultados.shape

In [ ]:
contagem_moa = dados_resultados.select_dtypes('int64').sum().sort_values(ascending = False)
contagem_moa

In [ ]:
contagem_moa = dados_resultados.select_dtypes('int64').sum(axis=1)
contagem_moa

In [ ]:
dados_resultados['n_moa'] =  dados_resultados.select_dtypes('int64').sum(axis=1)
dados_resultados['ativo_moa'] =  (dados_resultados['n_moa'] != 0)
dados_resultados.head()

In [ ]:
dados_combinados = pd.merge(dados, dados_resultados[['id','n_moa', 'ativo_moa']], on = 'id')
dados_combinados.head()

In [ ]:
dados_combinados.query('tratamento == "com_controle"')['ativo_moa'].value_counts()

In [ ]:
dados_combinados.query('tratamento == "com_droga"')['ativo_moa'].value_counts()

In [ ]:
composto_principal = dados_combinados['compostos'].value_counts().index[:5]
plt.figure(figsize=(12,8))
sns.boxplot(data = dados_combinados.query('compostos in @composto_principal'), y='g0', x ='compostos', hue='ativo_moa')

# Aula 5

In [ ]:
dados_combinados.head()

In [ ]:
dados_resultados.head()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

x = dados_combinados.select_dtypes('float64')
y = dados_combinados['ativo_moa']
x_treino,x_teste,y_treino, y_teste = train_test_split(x, y, stratify = y, test_size = 0.2, random_state=376)

modelo_rlogistica = LogisticRegression(max_iter=1000)
modelo_rlogistica.fit(x_treino, y_treino)
modelo_rlogistica.score(x_teste, y_teste) 

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

modelo_rlogistica = DummyClassifier('most_frequent')
modelo_rlogistica.fit(x_treino, y_treino)
previsao_dummy = modelo_rlogistica.predict(x_teste) 
accuracy_score(y_teste, previsao_dummy)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

x = dados_combinados.select_dtypes('float64')
y = dados_combinados['ativo_moa']
x_treino,x_teste,y_treino, y_teste = train_test_split(x, y, stratify = y, test_size = 0.2, random_state=376)

modelo_arvore = DecisionTreeClassifier(max_depth = 3)
modelo_arvore.fit(x_treino, y_treino)
modelo_arvore.score(x_teste, y_teste) 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

x = dados_combinados.select_dtypes('float64')
y = dados_combinados['ativo_moa']
x_treino,x_teste,y_treino, y_teste = train_test_split(x, y, stratify = y, test_size = 0.2, random_state=376)

teste=[]
treino=[]
for i in range(1, 15):
  modelo_arvore = DecisionTreeClassifier(max_depth = i)
  modelo_arvore.fit(x_treino, y_treino)
  teste.append(modelo_arvore.score(x_teste, y_teste)) 
  treino.append(modelo_arvore.score(x_treino, y_treino)) 

In [ ]:
sns.lineplot(x = range(1,15), y = teste, label = 'teste')
sns.lineplot(x = range(1,15), y = treino, label = 'treino')

In [ ]:
pd.get_dummies(dados_combinados['tratamento'])

In [ ]:
pd.get_dummies(dados_combinados, columns=['dose', 'tratamento', 'tempo'])

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import train_test_split

x = dados_combinados.drop(['id', 'n_moa', 'ativo_moa', 'compostos'], axis = 1)
x = pd.get_dummies(x, columns=['dose', 'tratamento', 'tempo'])
y = dados_combinados['ativo_moa']
x_treino,x_teste,y_treino, y_teste = train_test_split(x, y, stratify = y, test_size = 0.2, random_state=376)

modelo_floresta = RandomForestClassifier()
modelo_floresta.fit(x_treino, y_treino)
modelo_floresta.score(x_teste, y_teste) 